In [ ]:
%run _prepare.ipynb

# Geographic information (Notebook 4)

Maps are a great way to visualize geographc information, as they are intuitive as well as expressive.

The main types of maps used are:
 * **Symbols maps** (visualizing exact geographic locations)
 * **Choropleth maps** (visualizing areas with clear boundries)
 
Aside of these, there are also some rather special maps:
 * **Hexagon maps** (for voting districts)
 * **isarithmic** (for weather maps)
 * **Voronoi** (construct a choropleth-style mape using points)

As our dataset contains country aggreagtions and those have a clear area, we will only use choropleth maps.

## Using plotly

Plotly enables us the easily create a map for countries, as it does some intelligent lookup of countries iso-codes and maps those to positions within a world-map.

In [ ]:
px.choropleth(countries.sort_values("year"), 
    locations="iso_code", 
    color="co2", 
    hover_name="iso_code", 
    animation_frame="year",
   width=1000, height=550)


A great feature of plotly is its `animation_frame` feature, which allows us to easily 

In [ ]:
px.choropleth(countries.sort_values("year"), 
    locations="iso_code", 
    color="co2", 
    hover_name="iso_code", 
    animation_frame="year",
   width=1000, height=550)


# Projections

As our earth is a three dimensional sphere, it is impossible to project it to a two dimensional plane without loosing some information. The main aspects of geography are:

* shape of the continents / countries
* area
* angles
* distances
* directions

All projections need to make some compromises. Some popular examples:
* Mercator (used in Google maps): Drops the area proportions
* Exotisch: Lambert (keep size, not shape), Goode’s Homolsine (mix of conformal and equal area), 
* Gute Kompromisse: Mollweide (no rectangular form), Natural Earth, Robinson

For more info, read [this article](https://www.axismaps.com/guide/map-projections).

Generally it is important to note, that those aspects are less relevant for narrow visualiatzions (for example of a single city) and grow as we want to project the whole globe.

As all major charting libraries, plotly lets us choose which projection to use. It actually provides quite a high number of options:

In [ ]:
projections = ['equirectangular',  'mercator',  'orthographic',  'natural earth'] 
projections_other = ['kavrayskiy7',  'miller',  'robinson',  'eckert4',  'azimuthal equal area',  'azimuthal equidistant',  'conic equal area',  'conic conformal',  'conic equidistant',  'gnomonic',  'stereographic',  'mollweide',  'hammer',  'transverse mercator',  'albers usa',  'winkel tripel',  'aitoff', 'sinusoidal']

In [ ]:
for proj in projections:
    display(px.choropleth(countries.loc[filter_most_recent], 
        locations="iso_code", 
        color="co2", 
        hover_name="country", 
        projection=proj,
        title=f"World projected using {proj}",
        width=500, 
        height=500))

# Using Altair

Altair is also a good option for visualization geographic information. How ever, it does not do an automatic matching with the iso-codes within our dataset. Therefore we need to first get the map we want to show.

A common format to use is geo-json. You can download a projection of the world from [geojson-maps](https://geojson-maps.ash.ms/) and load it with geopandas

In [ ]:
import geopandas as gpd
#!wget https://github.com/datasets/geo-countries/raw/master/data/countries.geojson -O ../data/countries.geojson
geo_df = gpd.read_file('../data/countries.geojson')

In [ ]:
geo_df = geo_df.join(countries.loc[filter_most_recent].set_index("iso_code"), on="ISO_A3")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
world_map = alt.Chart(geo_df, width=900, height=380).mark_geoshape(stroke="white").encode(
    tooltip=["country:N", "continent:N"],
    color='co2:Q',
).project("equirectangular")

world_map

# Maps to select countries of interest

Maps combine greatly with Altairs interactivity and enables us to filter other charts based on our selection

In [ ]:
alt.Chart(countries, width=1000, height=250).mark_line().encode(
    x="year", 
    y="co2",
    color="country"
)

In [ ]:
# Create a selection
selected_countries = alt.selection_multi(on='click', fields=['country'], empty='none')

# Add the selection to the map
interactive_world = world_map.encode(color=alt.condition(selected_countries, alt.value('red'), 'co2_per_capita:Q')).add_selection(selected_countries)

# Creater a timeline that filters based on the selection
timeline = alt.Chart(countries, width=1000, height=250).mark_line().encode(
    x="year", 
    y="co2_per_capita",
    color="country"
).transform_filter(selected_countries)

interactive_world & timeline

# 📝 Task: Add some more charts, that link to the map

In [ ]:
def create_dashboard(target_column="co2", additional_timeseries=[]):
    selected_countries = alt.selection_multi(on='click', fields=['country'], empty='none')

    # Add the selection to the map
    interactive_world = world_map.encode(color=alt.condition(selected_countries, alt.value('red'), target_column)).add_selection(selected_countries)

    # Creater a timeline that filters based on the selection
    timelines = []
    for y in [target_column, *additional_timeseries]:
        timelines.append(
            alt.Chart(countries, width=1000, height=150).mark_line().encode(
                x="year", 
                y=y,
                color="country",
            ).transform_filter(selected_countries)
         )

    return interactive_world & alt.vconcat(*timelines)
    

In [ ]:
target_column = widgets.Dropdown(options=countries.columns.to_list(), value="co2_per_capita")
other_columns = widgets.SelectMultiple(options=countries.columns.to_list())

interact(create_dashboard, target_column=target_column, additional_timeseries=other_columns)